# Import neccesary libraries

In [3]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.collocations import ngrams

# Functions

In [3]:
import re
from unidecode import unidecode

stemmer = SnowballStemmer("spanish")
expections = ["@", r"\."]
stopwords = stopwords.words('spanish')

def clean_text(text):
    # ! EXCEPCIONES MANUALES!!!
    # Cast exceptions to "a"
    text = re.sub(r'@', 'a', text)
    # Cast from "." to ""
    text = re.sub(r'\.', '', text)
    
    # Trim the text
    text = text.strip()
    # Remove accents using unidecode, excluding 'ñ' and 'Ñ'
    text = ''.join(char if char in ('ñ', 'Ñ') else unidecode(char) for char in text)
    # Delete non-alphanumeric characters
    text = re.sub(r'[^\w\s]', ' ', text)
    # Delete additional spaces with regex
    text = re.sub(r'\s+', ' ', text).upper()
    return text

def filter_words(text):
    # Split the text by " "
    list_words = text.split(" ")
    # Delete empty strings
    list_words = list(filter(None, list_words))
    # Filter all stopwords
    return [word for word in list_words if word not in stopwords]

def get_list_stems(list_words):
    # Get stems
    return [stemmer.stem(word).upper() for word in list_words]

def get_n_gramas(list_stems, min_n_gramas, max_n_gramas):
    list_n_gramas = [" ".join(n_grama) for n in range(min_n_gramas, max_n_gramas + 1)
                     for n_grama in ngrams(list_stems, n)]
    return list_n_gramas

def calculate_forms(text, min_n_gramas=2, max_n_gramas=4):
    # First clean text
    text = clean_text(text)
    # Filter words
    list_words = filter_words(text)
    # Get list of stems
    list_stems = get_list_stems(list_words)
    # Get the n gramas
    list_n_gramas = get_n_gramas(list_stems, min_n_gramas, max_n_gramas)
    if list_n_gramas != []:
        return list_n_gramas
    else:
        return None

# Import collocations dictionary

In [4]:
df_collocations = pd.read_csv("./static/diccionario_collocation.csv")
list_collocations = df_collocations["FORMAS"].tolist()
df_collocations.head()

,LEMA,COLLOCATION,FORMAS
0,ABBYY FINEREADER,ABBYY FINEREADER,ABBYY FINEREAD
1,ABOGAR PERSONAS,ABOGAR POR OTRAS PERSONAS,ABOG PERSON
2,ASESORAR ABONOS PLANTAS,ASESORAR SOBRE ABONOS PARA PLANTAS,ABON PLANT
3,ABRASIVOS CHORRO,ABRASIVOS DE CHORRO,ABRAS CHORR
4,ABRIR CUENTAS BANCO,ABRIR CUENTAS DE BANCO,ABRIR CUENT


# Import offers

In [5]:
df_offers = pd.read_json("./data/descripcion_ofertas_infojobs_21_23.json")
df_offers.head()

,descripcion_oferta
0,Empresa de carácter internacional solicita adm...
1,Las funciones a desempeñar son las propias del...
2,Buscamos persona para puesto de Chófer -Repart...
3,Abrimos proceso de selección para dependiente ...
4,Tus funciones principales serán:\n\n- Atención...


# Get COLLOCATIONS from descripcion_oferta

In [7]:
def get_collocations(descripcion_oferta):
    # Get all forms from the description
    list_forms = calculate_forms(descripcion_oferta)
    # Find in the list of collocations
    list_collocations_found = [form for form in list_forms if form in list_collocations]
    # Get the LEMA from the collocations
    list_collocations_found = [df_collocations[df_collocations["FORMAS"] == form]["LEMA"].iloc[0] for form in list_collocations_found]
    return list_collocations_found

df_offers["COLLOCATIONS"] = df_offers["descripcion_oferta"].apply(get_collocations)

,descripcion_oferta,COLLOCATIONS
0,Empresa de carácter internacional solicita adm...,[]
1,Las funciones a desempeñar son las propias del...,[]
2,Buscamos persona para puesto de Chófer -Repart...,[BUSCAR PERSONAS]
3,Abrimos proceso de selección para dependiente ...,[]
4,Tus funciones principales serán:\n\n- Atención...,[]
5,¿Te apasionan los nuevos retos y quieres segui...,"[CREADOR BUEN AMBIENTE TRABAJO, FACILITAR INFO..."
6,En BCN Talent Logistics -la consultoría más fi...,[MERCANCIAS TRANSPORTE MARITIMO]
7,¿Te gustaría formar parte de una importante em...,"[MEJORAR SERVICIOS, GESTIONAR EQUIPO INTERMODAL]"
8,"CANARIAS SUBMARINE LINK, S.L.U. (CANALINK), op...",[]
9,¡Esta es tu oportunidad!\r\n\r\nEn Ecointegral...,"[PROCURAR DISTRIBUCION ELECTRICIDAD, RIESGOS L..."


# Save to compare the dataframe

In [ ]:
df_offers.to_json("./data/descripcion_ofertas_infojobs_21_23_collocations.json")

In [4]:
df_offers = pd.read_json("./data/descripcion_ofertas_infojobs_21_23_collocations.json")
df_offers.to_json("./data/descripcion_ofertas_infojobs_21_23_collocations.json", orient="records")